<a href="https://colab.research.google.com/github/gweaver96/journalclub1/blob/main/journalclub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Look, I made a thing to teach you about stuff. Let me explain.

In [ ]:
! git clone https://github.com/gweaver96/journalclub1

In [2]:
#Import all the stuff
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import pandas
pandas.options.mode.chained_assignment = None

There's loads of stuff here, but I'm going to pretend I know enough to explain it all.


The thing you just imported: **Tensorflow**, is the API that you use to write lots of different Machine Learning algorithms, and **Keras** is a more user-friendly wrapper on top of that. There are other APIs available, like PyTorch, but I've never used them and I'm scared of change. 



In [ ]:
# We're going to load the data now.
# I'm not going to explain this cause I'm guessing you understand how to do this already.

dataset = pandas.read_csv("/content/journalclub1/data.csv")
data = dataset.values
header = dataset.columns
dataset

# SO THEN

We have our data - the measurement of stinkiness at a range of distances from Walley's Quarry. After loading it all in, now we have a dataset that is made up of 3 columns;
\
1 - A useless numbered column going from 0 to 199 that I didn't know how to get rid of. \
2 - The distance in metres from the site. \
3 - The stinkiness rating, a scale in 'stinks per metre\^3' from 0 to 1, where 1 is the maximum stinkiness. \
\
We can plot the data on a graph below to see what it looks like. Again I can't be bothered explaining how matplotlib works.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15,10))
plt.scatter(dataset['Distance / metres (m)'], dataset['Stinkiness / stinks per metre^3 (st/m^3)'], s=10)
plt.grid(True)
plt.ylabel('Stinkiness / stinks per metre^3 (st/m^3)')
plt.xlabel('Distance / metres (m)')
plt.title('Stinkiness vs Distance from Walley\'s Quarry')
plt.show()

### Now we're doing stuff you might not have used before.

So in order to train our model we have to do 2 things. Firstly, we need to scale the data to between 0 and 1. Luckily, the scale of st/m^3 is already between 0 and 1, so we only have to do it for the distance measurements.
\
\
This scaling makes it easier for the stupid machine to learn properly, as all the features (inputs) and labels (outputs) are in a similar range.
There's a load of ways to do this in an annoying and complicated way but we're just going to scale it from minimum to maximum, between 0 and 1 with the MinMax scaler from sklearn.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = dataset['Distance / metres (m)']
y = dataset['Stinkiness / stinks per metre^3 (st/m^3)']

#setting the scale to not quite zero is good practice, because the stupid computer is too stupid to deal with values at the bounds
scale_x = MinMaxScaler(feature_range=(0.001,1.000))

#we make the x variable an array and reshape it into 2D for reasons literally impossible to understand
x_scaled = scale_x.fit_transform(np.array(x).reshape(-1,1))

plt.scatter(x_scaled, y, s=10)

Look! Now it's all between 0 and 1 almost!

Next we split the data into test and train sets. We will use the testing set later on to check if the model is good or not, and the training set is used to train the actual model we make.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.005, shuffle=True)

plt.figure(figsize = (15,10))
plt.scatter(x_train, y_train, c='k', s = 10, label = 'training data')
plt.scatter(x_test, y_test, c='r', s=10, label = 'test data')
plt.grid(True)
plt.legend()
plt.ylabel('Stinkiness / stinks per metre^3 (stm^-3)')
plt.xlabel('Scaled Distance')
plt.title('Stinkiness vs Distance from Walley\'s Quarry')
plt.show()

Now we do an actual model. Hold on to your seatbelts.

In [ ]:
#Building a Sequential model, with an output and an input layer, of one node each

model = Sequential()

model.add(Dense(units=1, input_dim=1))

model.add(Dense(units=1))

#compile it with a loss function and an optimiser
model.compile(loss=keras.losses.MeanAbsoluteError(), optimizer=keras.optimizers.Adam(learning_rate=0.01))
# this summary will tell us what the model is like
model.summary()

In [ ]:
#now we fit the model to the training data, over 20 epochs, and with 20% of the data used to validate training.
hist = model.fit(x_train, y_train, epochs=100, verbose=1, validation_split = 0.2)

Okay I don't know about you but I can't be bothered trying to figure out what's happening with all those numbers.

I need myself a great big lovely graph to show me what's going on. Let's plot the loss vs epochs below.

In [ ]:
def plot_loss(hist):
  plt.figure(figsize=(10,7))
  plt.plot(hist.history['loss'], label='loss')
  plt.plot(hist.history['val_loss'], label='val_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.grid(True)
plot_loss(hist)

In [ ]:
x_pred = np.linspace(0,1,10)
y_pred = model(x_pred)

In [ ]:
plt.figure(figsize = (15,10))

plt.plot(x_pred, y_pred, c='r', lw=2, label = 'model prediction')
plt.scatter(x_train, y_train, c='k', label = 'training data')

plt.grid(True)
plt.legend()
plt.ylabel('Stinkiness / stinks per metre^3 (stm^-3)')
plt.xlabel('Scaled Distance')
plt.title('Stinkiness vs Distance from Walley\'s Quarry')
plt.show()

Okay I mean maybe for George work that's a good fit, but for actual scientists we probably need to make a more complex model.

So let's do it.

Literally all we need to do is add more layers to the model, but this time we use activation functions to change the outputs of the layers.

In [ ]:
model = Sequential()

model.add(Dense(units=1, input_dim=1))

model.add(Dense(units=50, activation = 'ReLU'))

model.add(Dense(units=100, activation = 'ReLU'))

model.add(Dense(units=200, activation = 'ReLU'))

model.add(Dense(units=400, activation = 'ReLU'))

model.add(Dense(units=200, activation = 'ReLU'))

model.add(Dense(units=100, activation = 'ReLU'))

model.add(Dense(units=50, activation = 'ReLU'))

model.add(Dense(units=1))

model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001))

model.summary()


Hot dang it look at the number of trainable parameters on that bad boy.

Let's train it.

In [ ]:
hist = model.fit(x_train, y_train, epochs=1000, verbose=1, validation_split = 0.2)

In [ ]:
plot_loss(hist)

Okay looking at that loss plot, we're doing a thing called overfitting. Maybe I got too excited when I made a model that had way more trainable parameters as there are data points.

Run the next cell to see how the prediction looks. If there's loads of wiggly bits then we're definitely overfitting.

Go back and fix my shoddy work.

In [ ]:
x_pred = np.linspace(0,1,1000)
y_pred = model(x_pred)

plt.figure(figsize = (15,10))

plt.plot(x_pred, y_pred, c='r', linewidth=3, label = 'model prediction')
plt.scatter(x_train, y_train, c='k', label = 'training data')

plt.grid(True)
plt.legend()
plt.ylabel('Stinkiness / stinks per metre^3 (stm^-3)')
plt.xlabel('Distance')
plt.title('Stinkiness vs Distance from Walley\'s Quarry')
plt.show()

# 'Okay okay okay okay, but so what. I can do that way quicker with np.polyfit?' I can hear Joe shout.

Well well well, you poor, silly little lamb, let me show you the same thing, but in more than 2-D!

In [ ]:
prod_data = pandas.read_csv("/content/journalclub1/data2.csv")
prod_data

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(prod_data['x_pos'], prod_data['y_pos'], c=prod_data['prod'], cmap = 'RdYlGn', s=100)
plt.ylabel('Y Distance (m)')
plt.xlabel('X Distance (m)')
plt.title('Productivity with distance from Joe, Joe at (0,0)')
plt.colorbar(label = 'Productivity')
plt.show()

plt.figure(figsize = (15,10))
plt.scatter(prod_data['x_pos'], prod_data['y_pos'], c=prod_data['temp'], cmap = 'coolwarm', s=100)
plt.ylabel('Y Distance (m)')
plt.xlabel('X Distance (m)')
plt.title('Temperature with distance from Joe, Joe at (0,0)')
plt.colorbar(label = 'Temperature/Celsius')
plt.show()

In [ ]:
x = np.hstack((np.array(prod_data['x_pos']).reshape(-1,1), np.array(prod_data['y_pos']).reshape(-1,1), np.array(prod_data['temp']).reshape(-1,1)))
y = np.array(prod_data['prod']).reshape(-1,1)

#setting the scale to not quite zero is good practice, because the stupid computer is too stupid to deal with values at the bounds
scale = MinMaxScaler(feature_range=(0.001,0.999))
scaley = MinMaxScaler(feature_range=(0.001,0.999))

x_scaled = scale.fit_transform(np.array(x))
y_scaled = scaley.fit_transform(np.array(y))

plt.scatter(x_scaled[:,0], x_scaled[:,1], c=y_scaled, cmap = 'RdYlGn')
plt.colorbar()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size = 0.01, shuffle=True)

In [ ]:
model = Sequential()

model.add(Dense(units=3, input_dim=3))

#PUT YOUR MODEL HERE

model.add(Dense(units=1))

model.compile(loss= ..., optimizer=keras.optimizers.Adam(learning_rate=...))

model.summary()


In [ ]:
hist = model.fit(x_train, y_train, epochs=..., verbose=1, validation_split = 0.2)

In [ ]:
plot_loss(hist)

In [ ]:
y_pred = model(x_scaled)
y_pred = scaley.inverse_transform(y_pred)

yerr = y_pred - y

plt.figure(figsize = (15,10))
plt.scatter(prod_data['x_pos'], prod_data['y_pos'], c=y_pred, cmap = 'RdYlGn', s=100)
plt.ylabel('Y Distance (m)')
plt.xlabel('X Distance (m)')
plt.title('Productivity with distance from Joe, Joe at (0,0)')
plt.colorbar(label = 'Productivity')
plt.show()

plt.figure(figsize = (15,10))
plt.scatter(prod_data['x_pos'], prod_data['y_pos'], c=yerr, s=100)
plt.ylabel('Y Distance (m)')
plt.xlabel('X Distance (m)')
plt.title('Productivity with distance from Joe, Joe at (0,0)')
plt.colorbar(label = 'Productivity Error')
plt.show()

plt.figure(figsize = (15,10))
plt.scatter(prod_data['temp'], prod_data['prod'], c=yerr, s=100)
plt.ylabel('Temp/C')
plt.xlabel('Productivity')
plt.title('Productivity vs Temp')
plt.colorbar(label = 'Productivity Error')
plt.show()

plt.figure(figsize = (15,10))
plt.scatter(prod_data['prod'], y_pred, c=yerr, cmap = 'RdYlGn', s=100)
plt.ylabel('predicted productivity')
plt.xlabel('Productivity')
plt.title('Productivity vs Predictions')
plt.colorbar(label = 'Productivity Error')
plt.show()